# Mixed-effects: The Best Way
In order to improve upon the FFX approach, we need a method that accommodates the between-subjects variance. To see how to do this, let us return to our hierarchical model from earlier

$$
\begin{align}
y_{ij}  &= \mu_{j} + \epsilon_{ij} &\quad \text{(Level 1)} \\
\mu_{j} &= \mu + \eta_{j} &\quad \text{(Level 2)} \\
\end{align}
$$

If we collapse across the levels (by replacing $\mu_{j}$ with its equality from Level 2), we create a single model

$$
y_{ij}  = \mu + \eta_{j} + \epsilon_{ij}
$$

Importantly, this model contains a single parameter associated with the population mean (the $\mu$ term) and *two* error terms ($\eta_{j}$ and $\epsilon_{ij}$). Because the population means is considered a *constant* (i.e. it does not change from measurement-to-measurement), it is known as a *fixed-effect*. As such, any random variation in the value of $y_{ij}$ must come from $\eta_{j}$ and $\epsilon_{ij}$. As such, these terms are both *random variables*, with distributions assumed to have the following form

$$
\begin{align}
\eta_{j}       &\sim \mathcal{N}(0, \sigma^{2}_{b})     \\
\epsilon_{ij}  &\sim \mathcal{N}(0, \sigma^{2}_{w_{j}}) \\
\end{align}
$$

As such, these error terms capture both the *between-subject* and *within-subject* variances. Because they are random variables, they are termed *random-effects*. Because our model now contains both kinds of effects, it is known as a *mixed-effects* (MFX) model.

```{note}
In the world of fMRI, MFX analyses are often referred to as *random-effects* (RFX) models. This corresponds to the fact that we are treating subjects as a random draw from a population. The subjects are therefore seen as *random* rather than *fixed*. It is *not* implying that the model only contains random effects, though this would be the interpretation if this term was being used in the usual statistical sense. Just remember that RFX and MFX are used somewhat interchangeably in fMRI, just to make sure everyone is as confused as possible.
```

Importantly, collapsing the two levels together means that our overall probability model for an individual subject's data is given by

$$
y_{ij} \sim \mathcal{N}(\mu, \sigma^{2}_{b} + \sigma^{2}_{w_{j}}).
$$

Thus, each data-point we sample can be thought of as containing some mixture of the *within-subject* and the *between-subject* variance. This collapsed sampling model is illustrated in {numref}`complete-sampling-model-fig`.

```{figure} images/complete-sampling-model.png
---
width: 500px
name: complete-sampling-model-fig
---
Illustration of the complete sampling model where each observation (indicated by a cross) is drawn from a population distribution (dashed line) that contains the individual subject distributions (solid lines). The variance of each observation around the group mean is therefore a combination of the variance between the subjects and the variance within a subject.
```

The advantage of MFX models is that they can *separate* these two sources. This has several practical advantages
- The correct variance terms can be selected for testing different effects. For instance, effects associated with comparing groups need to use the *between-subject* variance, otherwise inference will be too liberal and we are more likely to detect false-positives.
- Subjects who are *noisy* (i.e. who have larger values of $\sigma^{2}_{w_{j}}$) can be *down-weighted* in the analysis. This means the model will automatically trust cleaner data sets and use the information from less-noisy subjects more than noisy subjects.

As such, MFX approaches are highly beneficial because they can separate and quantify these difference sources of variance, allowing for a model that is much more accurate and much more flexible.

## Computational Challenges for MFX
Although a MFX analysis is clearly advantageous, there are unfortunately computational challenges when it comes to using these methods with fMRI data. Collapsing all our data together to form one large model has some major requirements in terms of computing resources. Fitting that amount of data in memory at once may be possible for a small number of subjects, but it soon becomes impractical for larger sample sizes. 

One solution to this problem is lean on the hierarchical approach described earlier. Using this method, we estimate the GLM of each subject separately and then take contrasts of the parameter estimates from each subject through to a 2nd-level GLM analysis. These contrasts form the raw data at the 2nd-level, where we then estimate the group effects. This is shown in {numref}`multilevel-fig` from [Poldrack, Mumford and Nichols (2011)](https://www.librarysearch.manchester.ac.uk/discovery/fulldisplay?context=L&vid=44MAN_INST:MU_NUI&search_scope=MyInst_and_CI&tab=Everything&docid=alma992975905221601631). 

```{figure} images/multilevel-poldrack.png
---
width: 800px
name: multilevel-fig
---
Illustration of how the multi-level framework can be implemented in two stages when working with fMRI data.
```

In this example, subjects from two different groups took part in a task looking at pictures of `faces` and pictures of `houses`. The effect of interest for each subject is the difference in activation between `faces` and `houses`. This difference is calculated for each subject and then taken through to a group-level model, where the average difference for the first group is compared to the average difference for the second group. We will explain how this 2nd-level model is structured later. For now, just try and get a sense of the general concept here.

### The Problem of Single-subject Variances
The challenge for performing the analysis this way is taking the variance estimates for each subject through to the group-level. As it turns out, this is far from trivial and can be computationally very slow. Solutions to this problem have been presented by [Beckmann, Jenkinson & Smith (2003)](https://pubmed.ncbi.nlm.nih.gov/14568475/), who use a complicated Bayesian approach to combine the variance estimates alongside estimating the group-level effects. This is implemented as the `FLAME` algorithm in `FSL`. Another solution was presented more recently by [Chen *et al.* (2012)](https://pubmed.ncbi.nlm.nih.gov/22245637/), adapting a method from the world of meta-analyses to achieve similar results. This is implemented as `3dMEMA` in `AFNI`. Conspicuously, `SPM` is missing from this list. Now, it is unfair to say the `SPM` cannot do MFX, because there is a tool buried inside `SPM` for this (as described by [Friston *et al.*, 2005](https://pubmed.ncbi.nlm.nih.gov/22245637/)). However, this has never been presented as a major part of the `SPM` software. This is because the way that the `SPM` authors actually want you to do a group-level analyses is using a method known as the *summary statistics approach*, which will be the focus of the next section.